# Results Notebook

The aim of this notebook is to generate plots to showcase experiment results at `results/`

Following is plotted:
- SID plots of
    - Gradual - V1
        - SID score - Random graphs - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
        - SID score - BNlearn 3 datasets - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
        - Runtime - Random graphs - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
    - Gradual - V2 - Ablation
        - SID scores - Random graphs - ablation plots
        - SID scores - Random graphs - comparison to baselines, existing ABAPC, V2 best (least approx)
        - Runtime - Random graphs - ablation plots
        - Runtime - Random graphs - comparison to baselines, existing ABAPC, V2. May be combined with last plot

In [ ]:
import pandas as pd
import numpy as np

import importlib
import utils
importlib.reload(utils)
from utils import plot_runtime, double_bar_chart_plotly, process_model_names_and_runtime_v1_data, process_mean_std_sid_data, DAG_NODES_MAP, DAG_EDGES_MAP

## Small Graphs V1 and V2 Plots

In [ ]:
baselines_data_random = pd.read_csv('../../results/existing/random_graphs/all_existing_methods_metrics_cpdag.csv')
baselines_data_random_fgs = pd.read_csv('../../results/existing/random_graphs/fgs/all_existing_methods_metrics_cpdag.csv')
baselines_data_random = pd.concat([baselines_data_random, baselines_data_random_fgs], ignore_index=True)

baselines_data_bn = pd.read_csv('../../results/existing/bnlearn_graphs/all_existing_methods_metrics_cpdag.csv')
baselines_data_bn_fgs = pd.read_csv('../../results/existing/bnlearn_graphs/fgs/all_existing_methods_metrics_cpdag.csv')
baselines_data_bn = pd.concat([baselines_data_bn, baselines_data_bn_fgs], ignore_index=True)

v1_data_random = pd.read_csv('../../results/gradual/v1_random_graphs/cpdag_metrics.csv')
v1_data_bn = pd.read_csv('../../results/gradual/v1_bnlearn_graphs/cpdag_metrics.csv')

fact_sourcing_runtime_random = pd.read_csv('../../results/existing/runtime_sourcing_facts_random_graphs/runtime_results.csv')
fact_sourcing_runtime_bn = pd.read_csv('../../results/existing/runtime_sourcing_facts_bnlearn_graphs/runtime_results.csv')

v2_data_random = pd.read_csv('../../results/gradual/v2_random_graphs_3_to_6_nodes/cpdag_metrics.csv')
v2_data_bn = pd.read_csv('../../results/gradual/v2_bnlearn_graphs_up_to_6_nodes/cpdag_metrics.csv')


In [ ]:
v2_data_random.columns

In [ ]:
v2_data_random['model'] = 'V2'
v2_data_random['elapsed'] = (
    v2_data_random['elapsed_bsaf_creation'] +
    v2_data_random['elapsed_model_solution'] + 
    v2_data_random['aba_elapsed'] +
    v2_data_random['ranking_elapsed']
)

v2_data_bn['model'] = 'V2'
v2_data_bn['elapsed'] = (
    v2_data_bn['elapsed_bsaf_creation'] +
    v2_data_bn['elapsed_model_solution'] + 
    v2_data_bn['aba_elapsed'] +
    v2_data_bn['ranking_elapsed']
)

In [ ]:

v1_data_random['dataset'] = '|V|=' + v1_data_random['n_nodes'].astype(str) + ', |E|=' + v1_data_random['n_edges'].astype(str)
baselines_data_random['dataset'] = '|V|=' + baselines_data_random['n_nodes'].astype(str) + ', |E|=' + baselines_data_random['n_edges'].astype(str)
v2_data_random['dataset'] = '|V|=' + v2_data_random['n_nodes'].astype(str) + ', |E|=' + v2_data_random['n_edges'].astype(str)

In [ ]:
baselines_random_processed = process_mean_std_sid_data(baselines_data_random)
v1_data_with_model_names = process_model_names_and_runtime_v1_data(v1_data_random, fact_sourcing_runtime_random)
v1_random_processed = process_mean_std_sid_data(v1_data_with_model_names)
v2_data_random_processed = process_mean_std_sid_data(v2_data_random)

v1_random_plot_data = pd.concat([v1_random_processed, 
                                 baselines_random_processed[baselines_random_processed['n_nodes'] <= 6],
                                 v2_data_random_processed], 
                                 ignore_index=True)

In [ ]:
v1_random_plot_data.head(1)

In [ ]:
methods = [
    'Random',
    'FGS',
    'NOTEARS-MLP',
    'MPC',
    'ABAPC (Original)',
    'V1.1 Refined Fact Ranking',
    'V1.2 Model Selection by Refined Fact Strengths',
    'V1.3 Model Selection by Arrows Sum',
    'V1.4 Model Selection by Arrows Mean',
    'V2',
]
names_dict = {
    'Random': 'Random',
    'FGS': 'FGS',
    'NOTEARS-MLP': 'NOTEARS-MLP',
    'MPC': 'MPC',
    'ABAPC (Original)': 'ABAPC (Original)',
    'V1.1 Refined Fact Ranking': 'V1.1',
    'V1.2 Model Selection by Refined Fact Strengths': 'V1.2',
    'V1.3 Model Selection by Arrows Sum': 'V1.3',
    'V1.4 Model Selection by Arrows Mean': 'V1.4 ',
    'V2': 'V2',
}
colors_dict = {
        'Random': 'grey',
        'FGS': '#b85c00',
        'NOTEARS-MLP': '#9454c4',
        'MPC': '#379f9f',
        'ABAPC (Original)': '#0085CA',
        'V1.1 Refined Fact Ranking': "#9F8D00",
        'V1.2 Model Selection by Refined Fact Strengths': "#9E0000",
        'V1.3 Model Selection by Arrows Sum': "#ff34da",
        'V1.4 Model Selection by Arrows Mean': "#008100",
        'V2': "#ff8c00",
    }

In [ ]:
import importlib
import utils
importlib.reload(utils)
from utils import double_bar_chart_plotly

double_bar_chart_plotly(v1_random_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['n_sid_low', 'n_sid_high'],
                        names=['Best', 'Worst'],
                        labels=['Normalised SID = SID / Number of Edges in DAG', ''],
                        methods=methods,
                        dist_between_lines=0.1175,
                        intra_dis=0.112,
                        inter_dis=0.137,
                        lin_space=5,
                        nl_space=5,
                        start_pos = 0.03,
                            width=1300,
                            height=600,
                            range_y1=(0, 5),
                            range_y2=(0, 5))

In [ ]:
double_bar_chart_plotly(v1_random_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['precision', 'recall'],
                        names=['Precision', 'Recall'],
                        labels=['Precision', 'Recall'],
                        methods=methods,
                        dist_between_lines=0.1175,
                        intra_dis=0.112,
                        inter_dis=0.137,
                        lin_space=5,
                        nl_space=5,
                        start_pos = 0.03,
                            width=1300,
                            height=600,
                            range_y1=(0, 1),
                            range_y2=(0, 1))

In [ ]:
double_bar_chart_plotly(v1_random_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['f1', 'n_shd'],
                        names=['f1', 'n_shd'],
                        labels=['f1', 'n_shd'],
                        methods=methods,
                        dist_between_lines=0.1175,
                        intra_dis=0.112,
                        inter_dis=0.137,
                        lin_space=5,
                        nl_space=5,
                        start_pos = 0.03,
                            width=1300,
                            height=600,
                            range_y1=(0, 1.1),
                            range_y2=(0, 2))

Now bnlearn datasets for V1

In [ ]:
v1_data_bn['n_nodes'] = v1_data_bn['dataset'].map(DAG_NODES_MAP)
v1_data_bn['n_edges'] = v1_data_bn['dataset'].map(DAG_EDGES_MAP)
baselines_data_bn['n_nodes'] = baselines_data_bn['dataset'].map(DAG_NODES_MAP)
baselines_data_bn['n_edges'] = baselines_data_bn['dataset'].map(DAG_EDGES_MAP)
fact_sourcing_runtime_bn['n_nodes'] = fact_sourcing_runtime_bn['dataset'].map(DAG_NODES_MAP)
fact_sourcing_runtime_bn['n_edges'] = fact_sourcing_runtime_bn['dataset'].map(DAG_EDGES_MAP)
v2_data_bn['n_nodes'] = v2_data_bn['dataset'].map(DAG_NODES_MAP)
v2_data_bn['n_edges'] = v2_data_bn['dataset'].map(DAG_EDGES_MAP)

baselines_bn_processed = process_mean_std_sid_data(baselines_data_bn)
v1_bn_data_with_model_names = process_model_names_and_runtime_v1_data(v1_data_bn, fact_sourcing_runtime_bn)
v1_bn_processed = process_mean_std_sid_data(v1_bn_data_with_model_names)
v2_data_bn_processed = process_mean_std_sid_data(v2_data_bn)

v1_bn_plot_data = pd.concat([v1_bn_processed, 
                             baselines_bn_processed[baselines_bn_processed['n_nodes'] <= 6],
                             v2_data_bn_processed], 
                             ignore_index=True)

In [ ]:
v1_bn_plot_data.head(1)

In [ ]:
double_bar_chart_plotly(v1_bn_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['precision', 'recall'],
                        names=['Precision', 'Recall'],
                        labels=['Precision', 'Recall'],
                        methods=methods,
                        dist_between_lines=0.1565,
                        lin_space=6,
                        nl_space=6,
                        intra_dis = 0.161,
                        inter_dis = 0.174,
                        start_pos = 0.04,
                            width=1300,
                            height=600,
                            range_y1=(0, 1),
                            range_y2=(0, 1))

In [ ]:
double_bar_chart_plotly(v1_bn_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['f1', 'n_shd'],
                        names=['f1', 'n_shd'],
                        labels=['f1', 'n_shd'],
                        methods=methods,
                        dist_between_lines=0.1565,
                        lin_space=6,
                        nl_space=6,
                        intra_dis = 0.161,
                        inter_dis = 0.174,
                        start_pos = 0.04,
                            width=1300,
                            height=600,
                            range_y1=(0, 1.1),
                            range_y2=(0, 2))

In [ ]:
double_bar_chart_plotly(v1_bn_plot_data, 
                        names_dict,
                        colors_dict,
                        vars_to_plot=['n_sid_low', 'n_sid_high'],
                        names=['Best', 'Worst'],
                        labels=['Normalised SID = SID / Number of Edges in DAG', ''],
                        methods=methods,
                        dist_between_lines=0.1565,
                        lin_space=6,
                        nl_space=6,
                        intra_dis = 0.161,
                        inter_dis = 0.174,
                        start_pos = 0.04,
                            width=1300,
                            height=600,
                            range_y1=(0, 5),
                            range_y2=(0, 5))

Runtime for V1

In [ ]:
methods = [
    # 'Random',
    # 'FGS',
    # 'NOTEARS-MLP',
    # 'MPC',
    'ABAPC (Original)',
    'V1.1 Refined Fact Ranking',
    'V1.2 Model Selection by Refined Fact Strengths',
    'V1.3 Model Selection by Arrows Sum',
    'V1.4 Model Selection by Arrows Mean',
    'V2',
]

symbols_dict = {
    'ABAPC (Original)':'triangle-down-dot',
    'FGS':'triangle-up-dot',
    'NOTEARS-MLP':'pentagon-dot',
    'MPC':'hexagon2-dot',
    'Random':'x', 
    'V1.1 Refined Fact Ranking':'circle-dot',
    'V1.2 Model Selection by Refined Fact Strengths':'diamond-dot',
    'V1.3 Model Selection by Arrows Sum':'square-dot',
    'V1.4 Model Selection by Arrows Mean':'star-dot',
    'V2':'circle-dot'}

In [ ]:
v1_data_for_runtime_plot = pd.concat([v1_data_with_model_names, baselines_data_random, v2_data_random], ignore_index=True)

v1_data_for_runtime_plot = v1_data_for_runtime_plot.groupby(['n_nodes', 'n_edges', 'model'], as_index=False).aggregate(
    elapsed_mean=('elapsed', 'mean'),
    elapsed_std=('elapsed', 'std')
)

In [ ]:
v1_data_for_runtime_plot = v1_data_for_runtime_plot[v1_data_for_runtime_plot['n_nodes'] <= 6]
plot_runtime(v1_data_for_runtime_plot,
             colors_dict=colors_dict,
             names_dict=names_dict,
             symbols_dict=symbols_dict,
             methods=methods)